In [2]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
from scipy import stats
from tqdm.auto import tqdm

Продуктовый менеджер сформировал гипотезу, что новая версия экрана продаж будет
лучше перформить по сравнению со старой, в связи с тем, что на новом экране продаж
более понятно описаны преимущества платной версии приложения.
Продуктовый менеджер ожидает увеличение уровня конверсии из установки в покупку
платной версии приложения. Приложение тем временем уже очень активно
продвигается на рынке, каждый день привлекается в среднем по 30 000 новых
пользователей, текущий уровень конверсии из установки в покупку 5%.
Как бы вы предложили проверить данную гипотезу, какие инструменты и тесты бы
использовали в процессе проверки?

Чтобы проверить гипотезу о том, что при измененении версии экрана на новую вырастет конверсия из установки в покупку, нужно провести A/B тест, который сводится к сравнению средних двух выборок.

1. Дизайн теста
1.1. Определить метрику на которую будем влиять - конверсия из установки в покупку
Тип метрики - биномиальная
1.2. Подумать о метриках, которые мы можем испортить вводя изменение (guardrail metrics) - это может быть ARPU (если есть разные по стоимости версии приложения), retention (если приложение по подписке)
1.3. Посчитать MDE, уточнить у продакт менеджера, какое минимальное значение конверсии будет бизнес значимым.

MDE в % для конверсии в покупку равен 5.865, при количестве пользователей: 210000 (см. ниже)

Выяснили, что при заданных параметрах можем задетектировать изменение конверсии на 5.865%
Если такое изменение бизнес-значимо (обсуждаем с продактом), то начинаем тест
Если не бизнес-значимо, то можем:
 проводить тест дольше - например 2 недели, тогда MDE = 4.147 %
 увеличить alpha/beta

1.4. Провести симуляцию A/A/B тестов и замерить, что ошибки I и II рода не превышают alpha и beta соответственно

2. Сбор данных

3. Анализ собранных данных
3.1. Необходимо выбрать статистический тест для проверки гипотезы о сравнении средних двух выборок
С учетом того, что конверсия - биномиальная метрика и размер групп довольно большой
Нам подходит Z-test
3.2. На выходе из Z-test получим p-value
При заданной alpha = 0,05, смотрим p-val < 0.05 или нет

Если p-val < 0.05, то получили статистически значимый эффект на метрику (конверсию), выкатываем изменение
Если p-val > 0.05, то говорим о том, что нет статистически значимого эффекта на метрику, не выкатываем

3.3. Обсуждаем результаты с продакт менеджером, возможно вносим изменения в дизайн и запускаем повторный тест


Кол-во установок в день - 30т, в неделю 210т
Историческое среднее конверсии - 5%
Историческая дисперсия конверсии - sqrt(0,05*(1-0,05)) = 0,218
alpha = 95%
beta = 20%
отношение target/control групп = 50/50 - для наибольшей скорости проведения теста
Кол-во метрик за которыми следим - 2: конверсия + guardrail
Кол-во групп - 2: target, control

In [3]:
def get_minimal_determinable_effect_bernoulli(mu, sample_size, n_groups, n_metrics, r=1, compare='together', alpha=0.05, beta=0.2):
    """Возвращает MDE для биномиальной метрики, при заданных параметрах теста.

    mu: float, среднее выборки на исторических данных
    sample size: int, размер выборки для теста (включает в себя все группы)
    n_groups: int, количество групп в тесте
    n_metrics: int, количество целевых метрик в тесте, по которым будет приниатся решение
    r: float, отношение самой маленькой таргетной группы к контрольной группе
    compare: str, together - сравниваем все группы попарно, 'only_control' - только контрольную с тестовой

    return: MDE в %
    """
    if compare == 'together':
        comparisons = c_2_n(n_groups)
    elif compare == 'only_control':
        comparisons = n_groups - 1
    t_alpha = norm.ppf(1 - ((alpha / 2)/(comparisons*n_metrics)), loc=0, scale=1)
    t_beta = norm.ppf(1 - beta, loc=0, scale=1)
    disp_sum_sqrt = (mu* (1 - mu))**0.5
    sample_ratio_correction = (r+2+1/r)
    mde = np.sqrt(sample_ratio_correction)*(t_alpha + t_beta) * disp_sum_sqrt / np.sqrt(sample_size)
    return mde*100/mu

In [14]:
mean = 0.05 # Найдем среднее

sample_size = 210000 # Количество уникальных пользователей

metrics = 2 # Количество целевых метрик

groups = 2 # Количество групп

mde = get_minimal_determinable_effect_bernoulli(mean,sample_size,groups,metrics, compare = 'only_control')
print(f' MDE в % для конверсии в покупку равен {mde:0.3f}, при количестве пользователей: {sample_size}')

 MDE в % для конверсии в покупку равен 5.865, при количестве пользователей: 210000
